In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core.webservice import Webservice
from azureml.core.authentication import AzureCliAuthentication
import requests
import json

In [2]:
#Azure subscription
subscription_id = "" 

#Resource Group 
resource_group = "mlops-rg" 

#AML Workspace Name and associated Azure Region 
workspace_name = "mlops-aml-ws" 
workspace_region = "eastus2" 

#Webservice name, URL and key
webservice_name = "chd-preditcor-service" # <- the name used by Azure DevOps pipeline
webservice_url = "" 
webservice_key = "" 

In [3]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

Workspace configuration succeeded


### Call the deployed model

Since telemetry is not yet activated, no information will be recorded as a result of this call.

**Note**: These calls are just used to demonstrate the telemetry functionality. We don't care here about the inputs and outputs of the deployed model.

In [4]:
# This is dummy data, just to test the call
test_data = {"data":[[61, 1, 150, 103],[43, 1, 180, 99]]}

In [5]:
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ webservice_key)}
response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

Predictions
"[1, 0]"


### Activate telemetry

We are going to activate telemetry on the deployed model. Then we will make another call which will end up being logged by both Application Insights and data collection.

In [6]:
web_service = Webservice(ws, webservice_name)

In [7]:
web_service.update(enable_app_insights=True, collect_model_data=True)

Make a few calls that will be collected.

In [12]:
test_data = {"data":[[61, 1, 150, 103],[43, 1, 180, 99],[63,0,138,85]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ webservice_key)}
response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

Predictions
"[1, 0, 1]"
Predictions
"[1, 0, 1]"
Predictions
"[1, 0, 1]"


In [13]:
test_data = {"data":[[61, 1, 150, 103]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ webservice_key)}
response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

Predictions
"[1]"


In [14]:
test_data = {"data":[[43, 1, 180, 99]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ webservice_key)}
response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

Predictions
"[0]"


In [15]:
test_data = {"data":[[63,0,138,85]]}

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ webservice_key)}
response = requests.post(webservice_url, json.dumps(test_data), headers=headers)
print('Predictions')
print(response.text)

Predictions
"[1]"
